<a href="https://colab.research.google.com/github/ShiweiHe0713/TECH-GB_2336_Data-Science-for-Business-Techincal/blob/main/Regression_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# notebook created to get a multiple regression table for the advertising example

#Regression

Using regularization - Lasso and Ridge

In [2]:
#If opening in colab run this cell
!git clone https://github.com/CTVisMe/DS4B_Sp24.git
%cd DS4B_Sp24/ClassModules/Module5_ModelAssessment/

Cloning into 'DS4B_Sp24'...
remote: Enumerating objects: 837, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 837 (delta 113), reused 235 (delta 110), pack-reused 598
Receiving objects: 100% (837/837), 158.22 MiB | 12.01 MiB/s, done.
Resolving deltas: 100% (376/376), done.
Updating files: 100% (146/146), done.
/content/DS4B_Sp24/ClassModules/Module5_ModelAssessment


In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline



## Tayko data

Recall the Tayko dataset (from Shmueli book)

Many features to predict purchase - including catalogs and purchase infor

In [18]:
## lets look at regression for the Tayko data set

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, LinearRegression

from sklearn.metrics import mean_squared_error
# need to create new features with various powers of horsepower

path = "../Module2_Supervised/data/Tayko.csv"

df=pd.read_csv(path)
#df.info()

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   sequence_number       2000 non-null   int64
 1   US                    2000 non-null   int64
 2   source_a              2000 non-null   int64
 3   source_c              2000 non-null   int64
 4   source_b              2000 non-null   int64
 5   source_d              2000 non-null   int64
 6   source_e              2000 non-null   int64
 7   source_m              2000 non-null   int64
 8   source_o              2000 non-null   int64
 9   source_h              2000 non-null   int64
 10  source_r              2000 non-null   int64
 11  source_s              2000 non-null   int64
 12  source_t              2000 non-null   int64
 13  source_u              2000 non-null   int64
 14  source_p              2000 non-null   int64
 15  source_x              2000 non-null   int64
 16  source

In [5]:
# drop all cases where Purchase = 0 - we want to predict Spending when spending happens:

df_spend = df[df['Purchase'] != 0]

In [6]:
df_spend.describe()


,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
count,1000.0000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000
mean,963.5360,0.833000,0.197000,0.044000,0.036000,0.047000,0.134000,0.019000,0.013000,0.01200,...,0.017000,0.154000,2.077000,1975.010000,2475.384000,0.544000,0.514000,0.223000,1.0,205.249000
std,584.7072,0.373162,0.397931,0.205198,0.186383,0.211745,0.340823,0.136593,0.113331,0.10894,...,0.129336,0.361129,1.598633,1118.469756,1032.463509,0.498309,0.500054,0.416467,0.0,220.739559
min,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.0,3.000000
25%,436.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,1.000000,1027.750000,1790.750000,0.000000,0.000000,0.000000,1.0,94.750000
50%,951.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,2.000000,1995.000000,2612.500000,1.000000,1.000000,0.000000,1.0,153.000000
75%,1485.2500,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,2.250000,2912.000000,3270.000000,1.000000,1.000000,0.000000,1.0,233.750000
max,1997.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,15.000000,4188.000000,4188.000000,1.000000,1.000000,1.000000,1.0,1500.000000


## Linear Regression using statmodels and OLS

(OLS = ordinary least squares, or just vanilla linear regression)

In [9]:
#First fit a standard OLS model
#read in using read_csv
#from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS

X=df_spend.drop(["sequence_number","Purchase","Spending"],axis=1)
y=df_spend["Spending"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=1)

# We are going to use the OLS fitting method from statsmodels instead of sklearn.LinearRegression
# I like the way it outputs a nice regression table
#mlr_model=LinearRegression()
#mlr_model.fit(X_train,y_train)
X_train = sm.add_constant(X_train)
mlr_model=OLS(y_train,X_train).fit()

print(mlr_model.summary())



                            OLS Regression Results                            
Dep. Variable:               Spending   R-squared:                       0.438
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     2.729
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           0.000636
Time:                        00:14:49   Log-Likelihood:                -612.04
No. Observations:                 100   AIC:                             1270.
Df Residuals:                      77   BIC:                             1330.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  325.3416 

In [10]:
## print out RMSE


X_test = sm.add_constant(X_test)
y_pred=mlr_model.predict(X_test)
mlr_rmse=mean_squared_error(y_pred,y_test)

print(mlr_rmse.round(1))

31997.5


## Regularization : Ridge Regression

Ridge regression imposes an L2 (squared) penalty on the coefficients - "shrinking" them towards zero

In [11]:

for a in [0,1,2,3,4,5]:
  model = Ridge(alpha=a) # what we called lambda in class is alpha in the Ridge function

  model.fit(X_train, y_train)
  y_pred=model.predict(X_test)
  rmse=mean_squared_error(y_pred,y_test)
  print("lambda=",a,": rmse = ",rmse.round(1))



lambda= 0 : rmse =  31997.5
lambda= 1 : rmse =  30895.3
lambda= 2 : rmse =  30869.6
lambda= 3 : rmse =  30919.4
lambda= 4 : rmse =  30996.5
lambda= 5 : rmse =  31086.0


In [12]:
model_best=Ridge(alpha=2)

model_best.fit(X_train,y_train)

model_best.coef_

coef_table = zip(X.columns,model_best.coef_.round(1))

df3 = pd.DataFrame(coef_table, columns=['colname', 'coef'])

print(df3)

                 colname  coef
0                     US   0.0
1               source_a -29.3
2               source_c  11.6
3               source_b -51.6
4               source_d -14.9
5               source_e  27.6
6               source_m  10.8
7               source_o -29.9
8               source_h  22.2
9               source_r -38.0
10              source_s  -5.8
11              source_t -33.5
12              source_u -29.9
13              source_p  17.2
14              source_x -12.7
15              source_w -27.2
16                  Freq   8.3
17  last_update_days_ago  61.9
18   1st_update_days_ago  -0.0
19             Web order   0.0
20           Gender=male -34.1
21        Address_is_res -26.7


## Regularization : Lasso Regression

Lasso uses an L1 (or absolute value) penalty

In [13]:

# now lets do Lasso



for a in [0,1,2,3,4,5]:
  model = Lasso(alpha=a) # remember alpha here is lambda in our notes
  # fit model
  model.fit(X_train, y_train)
  y_pred=model.predict(X_test)
  rmse=mean_squared_error(y_pred,y_test)
  print("lambda=",a,": rmse = ",rmse.round(1))





<ipython-input-13-2f2a0eddad7a>:8: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.062e+05, tolerance: 2.158e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


lambda= 0 : rmse =  31997.5
lambda= 1 : rmse =  30706.2
lambda= 2 : rmse =  30994.9
lambda= 3 : rmse =  31304.0
lambda= 4 : rmse =  31506.2
lambda= 5 : rmse =  31678.5


In [14]:


model_best=Lasso(alpha=1)

model_best.fit(X,y)

coef_table = zip(X.columns,model_best.coef_.round(4))

df3 = pd.DataFrame(coef_table, columns=['colname', 'coef'])

print(df3)

                 colname     coef
0                     US  -8.7375
1               source_a  39.8079
2               source_c  -3.7640
3               source_b   0.0000
4               source_d  -0.0000
5               source_e  -0.0000
6               source_m  -0.0000
7               source_o   0.0000
8               source_h  -0.0000
9               source_r  58.6460
10              source_s  -0.0000
11              source_t  -0.0000
12              source_u  26.7984
13              source_p  -0.0000
14              source_x  -0.0000
15              source_w   0.0000
16                  Freq  89.9227
17  last_update_days_ago  -0.0133
18   1st_update_days_ago  -0.0136
19             Web order  -0.0000
20           Gender=male  -0.0000
21        Address_is_res -78.7675


## Preventing overfitting by variable selection

What if we had just select the best of the OLS features?

In [19]:
#What if we had just selected the best of the OLS features?

X=df_spend[["source_c","Freq","Address_is_res"]]
y=df_spend["Spending"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=1)

mlr_model=LinearRegression()
mlr_model.fit(X_train,y_train)
X_train = sm.add_constant(X_train)
mlr_model=OLS(y_train,X_train).fit()

print(mlr_model.summary())


                            OLS Regression Results                            
Dep. Variable:               Spending   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     16.83
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           7.30e-09
Time:                        00:22:32   Log-Likelihood:                -619.73
No. Observations:                 100   AIC:                             1247.
Df Residuals:                      96   BIC:                             1258.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             70.0901     21.842      3.

In [20]:

X_test = sm.add_constant(X_test)
y_pred=mlr_model.predict(X_test)
mlr_rmse=mean_squared_error(y_pred,y_test)

print(mlr_rmse.round(1))

30830.8
